In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [ ]:
df = pd.read_csv('BTC_Trial_NB.csv')
df.index = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df = df.drop(['Date', 'Month', 'Year', 'Network Difficulty'],axis=1)
df.head()

In [4]:
from sklearn import preprocessing

df_for_training=df[:1848]
df_for_testing=df[1848:]

scaler = preprocessing.MinMaxScaler()
df_for_training_scaled = scaler.fit_transform(df_for_training)

df_for_testing_scaled=scaler.transform(df_for_testing)

In [ ]:
df_for_training_scaled.shape, df_for_testing_scaled.shape

In [6]:
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,3])
    return np.array(dataX),np.array(dataY)

In [7]:
trainX,trainY=createXY(df_for_training_scaled,30)
testX,testY=createXY(df_for_testing_scaled,30)

In [ ]:
trainX.shape, trainY.shape, testX.shape, testY.shape

In [9]:
def build_model(units, dropout, lr, batch_size, epochs, dense):

 #LSTM Model
  model = Sequential()
  model.add(LSTM(abs(units), input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
  model.add(Dropout(abs(dropout)/100.0))
  model.add(LSTM(abs(units)))
  model.add(Dropout(abs(dropout)/100.0))
  model.add(Dense(abs(dense), activation='tanh'))
  model.add(Dense(1, activation='linear'))
  optimizer = Adam(learning_rate=abs(lr)/10000.0, amsgrad=False)

  # Compile the model
  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

  # Train the model
  model.fit(trainX, trainY, validation_split=0.2, epochs=abs(epochs), batch_size=abs(batch_size))

  return model


In [10]:
from keras import optimizers
from tensorflow.keras.optimizers import Adam
import math

def evaluate(individual):
  units, dropout, lr, batch_size, epochs, dense = individual

  units = int(individual[0])
  dropout = int(individual[1])
  lr = int(individual[2])
  batch_size = int(individual[3])
  epochs = int(individual[4])
  dense = int(individual[5])

  print('\nNum of neurons: ', units, '\nEpochs:', epochs,'\nBatch size:',batch_size,
        '\nLearning rate:', abs(lr)/10000.0,'\nDropout rate:', abs(dropout)/100.0, '\nDense:', dense)

  #LSTM Model
  model = Sequential()
  model.add(LSTM(abs(units), input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
  model.add(Dropout(abs(dropout)/100.0))
  model.add(LSTM(abs(units)))
  model.add(Dropout(abs(dropout)/100.0))
  model.add(Dense(abs(dense), activation='tanh'))
  model.add(Dense(1, activation='linear'))
  optimizer = Adam(learning_rate=abs(lr)/10000.0, amsgrad=False)

  # Compile the model
  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

  # Train the model
  model.fit(trainX, trainY, validation_split=0.2, epochs=abs(epochs), batch_size=abs(batch_size))

  # Evaluate the model
  loss, accuracy = model.evaluate(testX, testY, verbose=0)
  return loss,

In [11]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from datetime import datetime

In [12]:
import random
random.seed(17) 

# Setup DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Possible parameter values
units = range(16,129)
dropout = range(0,41)
lr = range(0,101)
batch_size = range(32,129)
epochs = range(20,51)
dense = range(8,65) 

#Define variables
toolbox.register("attr_units", random.choice, units)  # Number of LSTM units
toolbox.register("attr_dropout", random.choice, dropout)  # Dropout rate
toolbox.register("attr_lr", random.choice, lr)  # Learning rate
toolbox.register("attr_batch_size", random.choice, batch_size)  # Batch size
toolbox.register("attr_epochs", random.choice, epochs)  # Number of epochs
toolbox.register("attr_denses", random.choice, dense)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_units, toolbox.attr_dropout, toolbox.attr_lr,
                  toolbox.attr_batch_size, toolbox.attr_epochs, toolbox.attr_denses), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
#Define function
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

start_time = datetime.now()
print("Start time:", start_time)
hof = tools.HallOfFame(1)
population = toolbox.population(n=20)
algorithms.eaSimple(population, toolbox, cxpb = 0.7, mutpb = 0.2, ngen=20,
                    halloffame=hof, verbose=True)
best_individual = hof[0]
end_time = datetime.now()
print('End time:',end_time)
time_elapsed = end_time-start_time
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

In [ ]:
param_units = int(best_individual[0])         # Number of LSTM units
param_dropout = int(best_individual[1])       # Dropout rate
param_lr = int(best_individual[2])            # Learning rate
param_batch_size = int(best_individual[3])    # Batch size
param_epochs = int(best_individual[4])        # Number of epochs
param_denses = int(best_individual[5])        # Number of dense units

print(param_units, param_dropout/100,param_lr/10000,param_batch_size, param_epochs, param_denses)

In [ ]:
model = build_model(units=param_units,dropout=param_dropout, lr=param_lr,
            batch_size=param_batch_size, epochs=param_epochs,dense=param_denses)

In [ ]:
y_test_best_pred = model.predict(testX).flatten()
best_test_result = pd.DataFrame(data= {'Test Pred':y_test_best_pred, 'Actual':testY.flatten()})

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE_tuned = np.sqrt(mean_squared_error(testY, y_test_best_pred))
RMSE_tuned

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(best_test_result['Test Pred'], label='Predicted', color='red')
plt.plot(best_test_result['Actual'], label='Actual', color='blue')
plt.title('Close Price Values after tuning vs Actual Close Price')
plt.legend(loc='upper left')